In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-dark")
from matplotlib import cycler
import warnings
warnings.filterwarnings("ignore")
import ta
import seaborn as sns
import MetaTrader5 as mt5
from datetime import datetime
from UsefulFunctions import data

In [2]:
mt5.initialize()

True

In [5]:
# Fetching data from MetaTrade5 from 2010 to present using the custome function get_rates from the data.py file
df = data.get_rates("USDJPY", mt5.TIMEFRAME_H1)
df

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2010-01-01 06:00:00,93.000,93.000,93.000,93.000,77,30,0
2010-01-01 07:00:00,93.000,93.010,92.980,93.000,78,30,0
2010-01-01 08:00:00,93.000,93.000,93.000,93.000,79,30,0
2010-01-01 09:00:00,93.000,93.000,93.000,93.000,77,30,0
2010-01-01 10:00:00,93.000,93.000,93.000,93.000,77,30,0
...,...,...,...,...,...,...,...
2024-08-02 19:00:00,146.974,147.450,146.746,146.836,8855,16,0
2024-08-02 20:00:00,146.838,146.948,146.741,146.831,6358,18,0
2024-08-02 21:00:00,146.830,146.889,146.429,146.486,4225,16,0
